```
This software is part of GPU Ocean. 

Copyright (C) 2017-2019 SINTEF Digital
Copyright (C) 2017-2019 Norwegian Meteorological Institute

This notebook implements Swashes test 4.1.1 Dam break on wet domain 
without friction

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# SWASHES 4.1.1 Dam break on wet domain without friction

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors


import os
import datetime
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS,KP07, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
gpu_ctx = Common.CUDAContext()

In [ ]:
#Create output directory for images
#timestamp = datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
#imgdir='images_swashes_' + timestamp + "/"
#os.makedirs(imgdir)
#print ("Saving images to " + imgdir)

## Swashes dam break parameters 

Case 4.1.1 from O. Delestre et al., SWASHES: a compilation of shallow water
analytic solutions for hydraulic and environmental studies. International
Journal for Numerical Methods in Fluids, 72(3):269–300, 2013.

We use a 1D domain, meaning ny = 1.

### Plotting analytic solutions

In [ ]:
# Plot analytic solution
x0 = 5.0
L = 10.0
g = 9.81
hr = 0.001
hl = 0.005
eta_equilibrium = 0.003
mult_T = [0,1,2,3,4,5,6]
cm = 0.1578324867 # dependent of g, hr, hl

def xA(t, x0):
    return x0 - t*np.sqrt(g*hl)
def xB(t, x0):
    return x0 + t*(2*np.sqrt(g*hl) - 3*cm)
def xC(t, x0):
    return x0 + t*(2*cm**2 *(np.sqrt(g*hl) - cm))/(cm**2 - g*hr)

def h_rarefaction(t, x0, x):
    h =(4.0/(9.0*g))* (np.sqrt(g*hl) - (x - x0)/(2*t))**2
    u = (2.0/3.0)*((x-x0)/t + np.sqrt(g*hl))
    return h, u
    
def swashes_h(t, x0, x_array):
    h = np.zeros_like(x_array)
    u = np.zeros_like(x_array)
    N = len(x_array)
    x_a = xA(t, x0)
    x_b = xB(t, x0)
    x_c = xC(t, x0)
    for i in range(N):
        x = x_array[i]
        if x < x_a:
            h[i] = hl
        elif x < x_b:
            h[i], u[i] = h_rarefaction(t, x0, x)
        elif x < x_c:
            h[i] = cm*cm/g
            u[i] = 2*(np.sqrt(g*hl)  - cm)
        else:
            h[i] = hr
    return h, u

"""
Generates initial conditions for x-directional dam break at x0, 
and high level is given by hl, and low level by hr.
"""
def make1DSwashes(h, nx, ny, dx, dy, halo, x0, hl, hr):
    for j in range(0, ny+halo[0]+halo[2]):
        y = (j-halo[2])*dy
        for i in range(0, nx+halo[1]+halo[3]):
            x = (i-halo[3])*dx
            if ( x < x0):
                h[j,i] = hl
            else:
                h[j,i] = hr

# Plotting analaytic solutions to test the above functions
x_array = np.linspace(0,10.0,num=1000)
x_a = xA(6.0, x0)
x_b = xB(6.0, x0)
x_c = xC(6.0, x0)
print( x_a, x_b, x_c)
fig = plt.figure(figsize=(8,4))
for t in range(7):
    h, u = swashes_h(t, x0, x_array)
    plt.plot(x_array, h, label="t = " + str(t))
plt.plot(x0, hl, 'D')
plt.ylim(0.0, 0.006)
plt.legend(loc=0)
plt.title("Analytic solutions for $\eta$ at different $t$")

fig = plt.figure(figsize=(8,4))
for t in range(7):
    h, u = swashes_h(t, x0,  x_array)
    plt.plot(x_array, u, label="t = " + str(t))
plt.ylim(-0.1, 0.17)
plt.legend(loc=0)
plt.title("Analytic solutions for $u$ at different $t$")


# Simulation parameters:


In [ ]:
nx = 200
ny = 1
dx = L / (1.0*200)
dy = dx
print ("dx: ", dx)

theoretically_max_u = 0.15
max_dt = 0.25* dx/(theoretically_max_u + np.sqrt(g*hl))
print( "max_dt", max_dt)
dt = 0.01
T = 6.0
#T = 4.18


# parameters we don't use:
f = 0.0
r = 0.0
A = 0.1


# Run simulations

In [ ]:
## Non-staggered:

x_array = np.linspace(0.0,L,num=nx)
print (x_array[0], x_array[-1])

boundaryConditions = Common.BoundaryConditions()
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
print ("eta0.shape: ", eta0.shape)
make1DSwashes(eta0, nx, ny, dx, dy, ghosts, x0, hl, hr)
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Hi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

print( "T: ", T)
# Transform case so that we use eta and H instead of h and B:
eta0 = eta0 - eta_equilibrium
Hi = Hi + eta_equilibrium


# KP07
sim = KP07.KP07(gpu_ctx, \
                eta0, Hi, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                boundary_conditions=boundaryConditions)

t_kp07 = sim.step(T)
eta_kp07, hu_kp07, hv_kp07 = sim.download()

# CDKLM16
sim = CDKLM16.CDKLM16(gpu_ctx, \
                      eta0, u0, v0, Hi, \
                      nx, ny, \
                      dx, dy, dt, \
                      g, f, r, \
                      boundary_conditions=boundaryConditions)

t_cdklm = sim.step(T)
eta_cdklm, hu_cdklm, hv_cdklm = sim.download()


## Staggered CTCS
ghosts = [1,1,1,1] # north, east, south, west
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
h0 = np.ones(dataShape, dtype=np.float32, order='C') * eta_equilibrium;
eta0_ctcs = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');    
make1DSwashes(eta0_ctcs, nx, ny, dx, dy, ghosts, x0, hl, hr)
eta0_ctcs = eta0_ctcs - eta_equilibrium
ctcs_sim = CTCS.CTCS(gpu_ctx, \
                     h0, eta0_ctcs, u0, v0, \
                     nx, ny, dx, dy, dt, \
                     g, f, r, A, \
                     boundary_conditions=boundaryConditions )
#t = ctcs_sim.step(dt*370)
t_ctcs = ctcs_sim.step(T)
eta_ctcs, hu_ctcs, hv_ctcs = ctcs_sim.download()
print( "eta_ctcs.shape: ", eta_ctcs.shape)
print( "min, max (eta_ctcs): ", np.min(eta_ctcs), np.max(eta_ctcs))


## Staggered FBL
ghosts = [1,1,1,1]
#validDomain = [None, None, 0, 0]    
boundaryConditions = Common.BoundaryConditions()
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

H0 = np.ones(dataShape, dtype=np.float32, order='C') * eta_equilibrium;
eta0_fbl = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros((dataShape[0], dataShape[1]-1), dtype=np.float32, order='C');
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');

make1DSwashes(eta0_fbl, nx, ny, dx, dy, ghosts, x0, hl, hr)
eta0_fbl = eta0_fbl - eta_equilibrium
fbl_sim = FBL.FBL(gpu_ctx, \
                  H0, eta0_fbl, u0, v0, \
                  nx, ny, \
                  dx, dy, dt, \
                  g, f, r, \
                  boundary_conditions=boundaryConditions)
#t = ctcs_sim.step(dt*370)
t_fbl = fbl_sim.step(T)
eta_fbl, hu_fbl, hv_fbl = fbl_sim.download()
print( "eta_fbl.shape: ", eta_fbl.shape)
print( "min, max (eta_fbl): ", np.min(eta_fbl), np.max(eta_fbl))


## Analytic solution
eta_fasit, hu_fasit = swashes_h(t_kp07, x0, x_array)
print( "eta_fasit.shape: ", len(eta_fasit))

fig_eta = plt.figure(figsize=(8,4))
plt.plot(x_array, eta0[2,2:-2], "-.b", label="KP")
plt.plot(x_array, eta0[2, 2:-2], "--r", label="CDKLM")
plt.plot(x_array, eta0_ctcs[1, 1:-1], '--c', label="CTCS")
plt.plot(x_array, eta0_fbl[1,1:-1], ':m', label="FBL")
plt.plot(x_array, -H0[0,1:-1], 'k')

plt.ylim(-0.0035, 0.003)
plt.xlabel('$x$', fontsize=14)
plt.ylabel('$\eta$', fontsize=14)
plt.legend()
plt.title('Initial $\eta$')

### Plotting eta
fig_eta = plt.figure(figsize=(10,3))
plt.plot(x_array, eta0[2,2:-2], "g", label="Initial")
plt.plot(x_array, eta_fasit-eta_equilibrium, 'k', label="Analytic")
plt.plot(x_array, eta_kp07[2,2:-2], "-.b", label="KP")
plt.plot(x_array, eta_cdklm[2, 2:-2], "--r", label="CDKLM")
plt.plot(x_array, eta_ctcs[1, 1:-1], '--c', label="CTCS")
plt.plot(x_array, eta_fbl[1,1:-1], ':m', label="FBL")
plt.plot(x_array, -H0[0,1:-1], 'k')

#plt.plot(x_array, eta1[2,2:-2])
plt.ylim(-0.0035, 0.003)
plt.xlabel('$x$', fontsize=14)
plt.ylabel('$\eta$', fontsize=14)
plt.legend()
plt.title('$\eta$ at $t = $' + str(T))
plt.tight_layout()

print( "Times (fbl, ctcs, kp07, cdklm): ", t_fbl, t_ctcs, t_kp07, t_cdklm)

### Plotting hu

fig_hu = plt.figure(figsize=(10,3))
plt.plot(x_array, hu_fasit*eta_fasit, 'k', label="Analytic")
plt.plot(x_array, hu_kp07[2,2:-2], "-.b", label="KP")
plt.plot(x_array, hu_cdklm[2, 2:-2], "--r", label="CDKLM")
plt.plot(x_array, hu_ctcs[1, 1:-2], '--c', label="CTCS")
plt.plot(x_array, hu_fbl[0,1:], ':m', label="FBL")

plt.ylim(-0.00005, 0.00045)
plt.xlabel('$x$', fontsize=14)
plt.ylabel('$hu$', fontsize=14)
plt.title('$hu$ at $t = $' + str(T))
plt.legend()
plt.tight_layout()


### Save figures
#fig_eta.savefig(imgdir + "case411_" + timestamp + "_eta.png")
#fig_eta.savefig(imgdir + "case411_" + timestamp + "_eta.pdf")
#fig_hu.savefig(imgdir + "case411_" + timestamp + "_hu.png")
#fig_hu.savefig(imgdir + "case411_" + timestamp + "_hu.pdf")
#print("Figures saved in " + imgdir)


## Animation of FVM

In [ ]:

def sim_animation(sim_kp07, sim_cdklm, T, substep=10.0, skips=0):
    
    if skips > 0:
        t = sim_kp07.step(skips*substep)
        t = sim_cdklm.step(skips*substep)
    
    const_x = int(sim.nx/2)
    eta_kp07, hu_kp07, hv_kp07 = sim_kp07.download(interior_domain_only=True)
    eta_cdklm, hu_cdklm, hv_cdklm = sim_cdklm.download(interior_domain_only=True)
    Hm = sim_kp07.downloadBathymetry()[1]
    simH_xsec = Hm[2:-2, const_x]

    
    #Create figure and plot initial conditions
    #fig = plt.figure(figsize=(12, 15))
    
    fig = plt.figure(figsize=(12,10))

    gs = fig.add_gridspec(2,1)
    ax_eta = fig.add_subplot(gs[0, :])
    ax_hu  = fig.add_subplot(gs[1, :])
    
    x_array = np.linspace(0.0,L,num=nx)
    
    ## Analytic solution
    eta_fasit, hu_fasit = swashes_h(sim_kp07.t, x0, x_array)
    
    sp_eta_bathymetry, = ax_eta.plot(x_array, -H0[0,1:-1], 'k')
    sp_eta_fasit, = ax_eta.plot(x_array, eta_fasit-eta_equilibrium, 'k', label="Analytic")
    sp_eta_kp,    = ax_eta.plot(x_array, eta_kp07[0,:], "-.b", label="KP")
    sp_eta_cdklm, = ax_eta.plot(x_array, eta_cdklm[0,:], "--r", label="CDKLM")

    #plt.plot(x_array, eta1[2,2:-2])
    ax_eta.set_ylim(-0.0035, 0.003)
    ax_eta.set_xlabel('$x$', fontsize=14)
    ax_eta.set_ylabel('$\eta$', fontsize=14)
    ax_eta.legend()
    ax_eta.set_title('$\eta$ at $t = $' + str(T))
    
    sp_hu_fasit, = ax_hu.plot(x_array, hu_fasit*eta_fasit, 'k', label="Analytic")
    sp_hu_kp,    = ax_hu.plot(x_array, hu_kp07[0,:], "-.b", label="KP")
    sp_hu_cdklm, = ax_hu.plot(x_array, hu_cdklm[0,:], "--r", label="CDKLM")

    #plt.plot(x_array, eta1[2,2:-2])
    ax_hu.set_ylim(-0.00005, 0.00045)
    ax_hu.set_xlabel('$x$', fontsize=14)
    ax_hu.set_ylabel('$hu$', fontsize=14)
    ax_hu.set_title('$hu$ at $t = $' + str(T))
    ax_hu.legend()
    
    #Helper function which simulates and plots the solution
    def animate(i):
        if (i>0):
            t = sim_kp07.step(substep)
            t = sim_cdklm.step(substep)
        else:
            t = 0.0
        eta_kp07, hu_kp07, hv_kp07 = sim_kp07.download(interior_domain_only=True)
        eta_cdklm, hu_cdklm, hv_cdklm = sim_cdklm.download(interior_domain_only=True)
        
        eta_fasit, hu_fasit = swashes_h(sim_kp07.t, x0, x_array)

        sp_eta_fasit.set_ydata(eta_fasit-eta_equilibrium)
        sp_eta_kp.set_ydata(eta_kp07[0,:])
        sp_eta_cdklm.set_ydata(eta_cdklm[0,:])
        
        sp_hu_fasit.set_ydata(hu_fasit*eta_fasit)
        sp_hu_kp.set_ydata(hu_kp07[0,:])
        sp_hu_cdklm.set_ydata(hu_cdklm[0,:])
        
        
        fig.suptitle("Time = {:04.0f} s ".format(t), fontsize=18)
        print(".", end='')
        

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(fig)
    
    return anim

In [ ]:
boundaryConditions = Common.BoundaryConditions()
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
print ("eta0.shape: ", eta0.shape)
make1DSwashes(eta0, nx, ny, dx, dy, ghosts, x0, hl, hr)
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Hi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

print( "T: ", T)
# Transform case so that we use eta and H instead of h and B:
eta0 = eta0 - eta_equilibrium
Hi = Hi + eta_equilibrium


# KP07
sim_kp07 = KP07.KP07(gpu_ctx, \
                eta0, Hi, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                boundary_conditions=boundaryConditions)

# CDKLM16
sim_cdklm = CDKLM16.CDKLM16(gpu_ctx, \
                      eta0, u0, v0, Hi, \
                      nx, ny, \
                      dx, dy, dt, \
                      g, f, r, \
                      boundary_conditions=boundaryConditions)


sim_animation(sim_kp07, sim_cdklm, 60, substep=0.1, skips=0)